In [27]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import datetime

#Capture time = 2 min
# Tempo de envio 500ms 

SAMPLING_SPACING = 4

data_list = []
FILE = f"SAMPLING_SPACING_{SAMPLING_SPACING}"

# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
# Callback chamado quando uma mensagem PUBLISH é recebida do servidor.
def on_message(client, userdata, msg):
    global data_list
    global now
    data = json.loads(msg.payload)
    data_list.append(data)
    print(data_list[-1])
    if(now + datetime.timedelta(seconds=120) < datetime.datetime.now() ):
        raise(Exception)

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.hivemq.com", 1883, 60)

try:
    now = datetime.datetime.now()
    client.loop_forever()
except BaseException:
    with open(f"./data/{FILE}.json","w") as file:
        #pass
        json.dump(data_list,file)

Conectado com o código de resultado 0
[{'gateway': 'ac233fc09283', 'timestamp': '2024-09-24T09:28:32.049Z', 'locator': [True, False, False, False]}, {'mac': 'c300000a7a63', 'timestamp': '2024-09-24T09:28:32.127Z', 'no': 0, 'rssi': -46, 'raw': '0201061bff3906cafb011400f5ffe700470000000000000000e721e4c477ea', 'aoa': {'frequency': 2426, 'iq': [-91, 151, -147, -114, 127, -126, 93, 153, -170, 53, -25, -183, 180, 4, -37, 180, 71, -113, -119, 39, 75, -23, -133, -25, 72, 100, -48, -62, 31, 135, 62, -113, -37, 67, 108, -76, -126, -9, 78, 6, -120, -68, 31, 121, -17, -75, 10, 140, 90, -90, -62, 46, 137, -45, -117, -47, 71, 33, -90, -109, -4, 127, 2, -77, -67, 120, 114, -51, -72, 28, 140, 2, -91, -87, 51, 54, -57, -125, -40, 115, 31, -71, -100, 100]}}]
[{'gateway': 'ac233fc09283', 'timestamp': '2024-09-24T09:28:36.049Z', 'locator': [True, False, False, False]}, {'mac': 'c300000a7a63', 'timestamp': '2024-09-24T09:28:36.140Z', 'no': 0, 'rssi': -45, 'raw': '0201061bff3906cafb0114f500e7ff4700000000000

In [28]:
from matplotlib import pyplot as plt

def ref_sampler(iq: list):
    data = []
    for i in range(0,16,2):
        data.append(np.angle(complex(iq[i],iq[i+1])))
    return np.array(data)

def pm(x:float):
    #Keeps the angle between +- pi
    return (x + np.pi) % (2 * np.pi) - np.pi #coomentg - np.pi to keep it between 0 => 2pi

data = ref_sampler( data_list[0][1]["aoa"]["iq"])
data

array([ 2.11316135, -2.48196391, -0.78144561,  1.02462976,  2.8393778 ,
       -1.70656788,  0.02221857,  1.77352802])

In [30]:
# we will always do ahead - before
pstep = np.rad2deg((data[1:]-data[:-1]))
pstep


array([-263.28128378,   97.43252135,  103.48049665,  103.97740349,
       -260.46350168,   99.05216721,  100.3426401 ])